In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

# Define the Transformer Classification Model
class TransformerClassificationModel(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_encoder_layers=2, num_decoder_layers=2, num_classes=4):
        super(TransformerClassificationModel, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)  # Embedding Layer
        self.positional_encoding = nn.Parameter(torch.zeros(1, d_model))

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers)

        self.decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_decoder_layers)

        self.fc = nn.Linear(d_model, num_classes)  # Output layer for the number of classes
        self.softmax = nn.Softmax(dim=1)  # Softmax for multi-class classification

    def forward(self, src, tgt):
        # Add positional encoding and apply embedding
        src = self.embedding(src) + self.positional_encoding
        tgt = self.embedding(tgt) + self.positional_encoding

        # Ensure the input is 3D for permute (batch_size, d_model) -> (batch_size, seq_len=1, d_model)
        src = src.unsqueeze(1) if src.dim() == 2 else src
        tgt = tgt.unsqueeze(1) if tgt.dim() == 2 else tgt

        # Ensure that inputs are 3D before permuting
        if src.dim() == 4:
            src = src.squeeze(1)  # Remove extra dimension if present
        if tgt.dim() == 4:
            tgt = tgt.squeeze(1)  # Remove extra dimension if present

        # Permute to fit Transformer input format (seq_len=1, batch_size, d_model)
        src = src.permute(1, 0, 2)  # (seq_len=1, batch_size, d_model)
        tgt = tgt.permute(1, 0, 2)  # (seq_len=1, batch_size, d_model)

        # Transformer encoding and decoding
        memory = self.encoder(src)
        output = self.decoder(tgt, memory)

        # Reshape and apply final fully connected layer
        output = output.permute(1, 0, 2).squeeze(1)

        # Pass through the fully connected layer
        output = self.fc(output)  # Now the output has shape (batch_size, num_classes)

        # Apply softmax to get class probabilities
        output = self.softmax(output)

        return output  # Probability for each class


# Load data
data = pd.read_csv('health_data.csv')

# Filter out rows where Status contains invalid or unknown values
valid_statuses = ["Low", "Medium", "High", "Above High"]
data = data[data['Status'].isin(valid_statuses)]  # Keep only rows with valid statuses

# Encode the Gender column using LabelEncoder
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

# Encode the Disease column
label_encoder_disease = LabelEncoder()
data['Disease'] = label_encoder_disease.fit_transform(data['Disease'])

# Map the Status column to numerical values
status_mapping = {"Low": 0, "Medium": 1, "High": 2, "Above High": 3}
data['Status'] = data['Status'].map(status_mapping)

# Select the feature and target columns
X = data[['Gender', 'Age', 'Height (cm)', 'Weight (kg)', 'BMI', 'Disease']]
y = data['Status']

# Scale the feature columns
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert the data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Change to long for classification
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)  # Change to long for classification

# Create PyTorch Datasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model
input_dim = X_train.shape[1]  # Number of features in the input
model = TransformerClassificationModel(input_dim)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs, inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        running_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

# Save the trained model
torch.save(model.state_dict(), 'multi_class_model1.pth')
print("Model saved successfully.")





D:\anaconda\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10/100, Loss: 1.2539039714336395
Epoch 20/100, Loss: 1.23795349316597
Epoch 30/100, Loss: 1.2240650898456573
Epoch 40/100, Loss: 1.1971054473638534
Epoch 50/100, Loss: 1.192634397006035
Epoch 60/100, Loss: 1.1910223511219025
Epoch 70/100, Loss: 1.194203799510002
Epoch 80/100, Loss: 1.1909582775354386
Epoch 90/100, Loss: 1.1910071989774704
Epoch 100/100, Loss: 1.1909530151367187
Model saved successfully.


D:\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
C:\Users\binuo\AppData\Local\Temp\ipykernel_9032\183361792.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `

Predicted Status: 0


In [10]:
# Test with a single input value
# Example single input value (ensure the format is correct)
single_input = np.array([[0, 25, 180, 75, 40.1, 2]])  # Example input

# Scale the input using the same scaler
single_input_scaled = scaler.transform(single_input)

# Convert the scaled input to a tensor
single_input_tensor = torch.tensor(single_input_scaled, dtype=torch.float32)

# Load the trained model for testing
model = TransformerClassificationModel(input_dim)
model.load_state_dict(torch.load('multi_class_model.pth'))
model.eval()  # Set model to evaluation mode

# Pass the input through the model
with torch.no_grad():  # No gradients needed for testing
    single_input_tensor = single_input_tensor.unsqueeze(0)  # Add batch dimension
    output = model(single_input_tensor, single_input_tensor)  # Forward pass

# Get the predicted class
predicted_class = torch.argmax(output, dim=1)
print(f'Predicted Status: {predicted_class.item()}')

Predicted Status: 0


D:\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
C:\Users\binuo\AppData\Local\Temp\ipykernel_9032\606346334.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `w

In [11]:
# Example multiple inputs (ensure the format is correct)
multiple_inputs = np.array([[1, 25, 180, 75, 23.1, 2], 
                            [0, 35, 165, 60, 22.0, 1], 
                            [1, 45, 170, 85, 29.4, 3]])  # Example input batch

# Scale the inputs using the same scaler
multiple_inputs_scaled = scaler.transform(multiple_inputs)

# Convert the scaled inputs to a tensor
multiple_inputs_tensor = torch.tensor(multiple_inputs_scaled, dtype=torch.float32)

# Load the trained model for testing
model = TransformerClassificationModel(input_dim)
model.load_state_dict(torch.load('multi_class_model1.pth'))
model.eval()  # Set model to evaluation mode

# Pass the inputs through the model
with torch.no_grad():  # No gradients needed for testing
    multiple_inputs_tensor = multiple_inputs_tensor.unsqueeze(1)  # Add batch dimension
    output = model(multiple_inputs_tensor, multiple_inputs_tensor)  # Forward pass

# Get the predicted classes for the batch
predicted_classes = torch.argmax(output, dim=1)
print(f'Predicted Statuses for the batch: {predicted_classes}')

Predicted Statuses for the batch: tensor([0, 3, 1])


D:\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
C:\Users\binuo\AppData\Local\Temp\ipykernel_9032\61995748.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `we

In [12]:
# Example of 20 inputs (samples) with features: [Gender, Age, Height (cm), Weight (kg), BMI, Disease]
multiple_inputs = np.array([
    [1, 25, 180, 75, 23.1, 2], 
    [0, 35, 165, 60, 22.0, 1], 
    [1, 45, 170, 85, 29.4, 3], 
    [0, 30, 175, 70, 22.9, 0], 
    [1, 50, 160, 55, 21.5, 1], 
    [0, 40, 185, 90, 26.3, 2], 
    [1, 28, 155, 50, 20.8, 3], 
    [0, 33, 178, 78, 24.6, 0], 
    [1, 22, 172, 68, 22.9, 2], 
    [0, 27, 168, 64, 22.7, 1], 
    [1, 55, 182, 88, 26.6, 0], 
    [0, 38, 162, 58, 22.1, 1], 
    [1, 47, 177, 80, 25.5, 2], 
    [0, 29, 174, 72, 23.8, 3], 
    [1, 34, 169, 66, 23.1, 0], 
    [0, 31, 170, 67, 23.2, 1], 
    [1, 53, 178, 83, 26.2, 2], 
    [0, 44, 164, 59, 21.9, 3], 
    [1, 26, 176, 74, 23.9, 0], 
    [0, 36, 167, 61, 22.0, 1]
])

# Scale the inputs using the same scaler
multiple_inputs_scaled = scaler.transform(multiple_inputs)

# Convert the scaled inputs to a tensor
multiple_inputs_tensor = torch.tensor(multiple_inputs_scaled, dtype=torch.float32)

# Load the trained model for testing
model = TransformerClassificationModel(input_dim)
model.load_state_dict(torch.load('multi_class_model1.pth'))
model.eval()  # Set model to evaluation mode

# Pass the inputs through the model
with torch.no_grad():  # No gradients needed for testing
    multiple_inputs_tensor = multiple_inputs_tensor.unsqueeze(1)  # Add batch dimension
    output = model(multiple_inputs_tensor, multiple_inputs_tensor)  # Forward pass

# Get the predicted classes for the batch
predicted_classes = torch.argmax(output, dim=1)
print(f'Predicted Statuses for the batch: {predicted_classes}')


Predicted Statuses for the batch: tensor([0, 3, 1, 1, 3, 0, 1, 1, 0, 3, 1, 3, 0, 1, 1, 3, 0, 1, 1, 3])


D:\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
C:\Users\binuo\AppData\Local\Temp\ipykernel_9032\2001406042.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `